In [1]:
import pandas as pd
import os
os.chdir("/home/yangkai/codegen-detection/")
hum_data = pd.read_pickle("dataset/apps_hum_test.pkl")
gen_data = pd.read_pickle("dataset/apps_gen_test.pkl")

In [11]:
import json
def load_json_dataset(pth):
    dataset = []
    cnt = 0
    with open(pth,"r") as f:
        for line in f:
            try:
                dataset.append(json.loads(line.strip()))
                cnt += 1
            except:
                dataset.append([{},{}])

    return dataset

gen_nocot_rewrite = load_json_dataset("prompt_gpt/mbpp_gen_test_gpt_resample2.jsonl")
hum_nocot_rewrite = load_json_dataset("prompt_gpt/mbpp_hum_test_gpt_resample2.jsonl")

In [12]:
import re
def extract_gpt_code(resp,id=0):
    try:
        text = resp['choices'][id]['message']['content']
        # text = resp[1][0]['generated_text']
    except KeyError:
        return ''
    pattern = r'```python\n(.+?)\n```'
    code_blocks = re.findall(pattern, text, re.DOTALL)
    if len(code_blocks) > 0:
        return code_blocks[0]

    if resp['choices'][id]['finish_reason'] == 'length':
        pattern2 = r'```python\n(.+?)'
    else:
        pattern2 = r'``` python\n(.+?)\n```'

    # pattern2 = r'```python\n(.+?)'
    code_blocks = re.findall(pattern2, text, re.DOTALL)
    if len(code_blocks) > 0:
        return code_blocks[0]

    pattern3 = r"```\n(.+?)\n```"
    code_blocks = re.findall(pattern3, text, re.DOTALL)
    if len(code_blocks) > 0:
        return code_blocks[0]

    if resp['choices'][id]['finish_reason'] == 'length':
        pattern4 = r"```\n(.+?)"
        code_blocks = re.findall(pattern4, text, re.DOTALL)
        if len(code_blocks) > 0:
            return code_blocks[0]
    
    pattern5 = r"Python\n(.+?)"
    code_blocks = re.findall(pattern5, text, re.DOTALL)
    if len(code_blocks) > 0:
        return code_blocks[0]

    return ""

failed_case = 0
for resp in gen_nocot_rewrite:
    out = extract_gpt_code(resp,id=0)
    if len(out) == 0:
        failed_case += 1
        # print(resp)
failed_case

0

In [13]:
for idx in [2,3,4,5,6,7,8]:
    key_name = f'gpt_resample_code{idx}'
    hum_data[key_name] = hum_nocot_rewrite
    hum_data[key_name] = hum_data[key_name].apply(lambda x: extract_gpt_code(x,idx-2))

    gen_data[key_name] = gen_nocot_rewrite
    gen_data[key_name] = gen_data[key_name].apply(lambda x: extract_gpt_code(x,idx-2))

    from prepro.code_prepro import CommentsRemover,PygmentsTokenizer,EmptyLinesRemover
    tokenizer = PygmentsTokenizer('Python')
    comment_remover = CommentsRemover(tokenizer)
    emptyline_remover = EmptyLinesRemover()

    def pipeline(x):
        return emptyline_remover.filter(comment_remover.filter(x))

    cleaned_name = f'gpt_resample_code_cleaned{idx}'
    gen_data[cleaned_name] = gen_data[key_name].apply(pipeline)
    hum_data[cleaned_name] = hum_data[key_name].apply(pipeline)

In [3]:
# check empty
for idx in [3,4,5,6,7,8]:
    cleaned_name = f'gpt_resample_code_cleaned{idx}'
    print((gen_data[cleaned_name].apply(len) == 0).sum())
    print((hum_data[cleaned_name].apply(len) == 0).sum())


23
8
23
6
20
6
21
8
22
10
21
8


In [9]:
hum_data.to_pickle("dataset/mbpp_hum_test.pkl")
gen_data.to_pickle("dataset/mbpp_gen_test.pkl")

In [4]:
sampled_hum_data = pd.read_pickle("dataset/apps_hum_test100.pkl")
sampled_gen_data = pd.read_pickle("dataset/apps_gen_test100.pkl")

In [6]:
sampled_gen_data['gpt_resample_code_cleand3'] = gen_data['gpt_resample_code_cleaned3'][gen_data['problem_id'].isin(sampled_gen_data.problem_id)]

sampled_gen_data['gpt_resample_code_cleand4'] = gen_data['gpt_resample_code_cleaned4'][gen_data['problem_id'].isin(sampled_gen_data.problem_id)]

sampled_gen_data['gpt_resample_code_cleand5'] = gen_data['gpt_resample_code_cleaned5'][gen_data['problem_id'].isin(sampled_gen_data.problem_id)]

sampled_gen_data['gpt_resample_code_cleand6'] = gen_data['gpt_resample_code_cleaned6'][gen_data['problem_id'].isin(sampled_gen_data.problem_id)]

sampled_gen_data['gpt_resample_code_cleand7'] = gen_data['gpt_resample_code_cleaned7'][gen_data['problem_id'].isin(sampled_gen_data.problem_id)]

sampled_gen_data['gpt_resample_code_cleand8'] = gen_data['gpt_resample_code_cleaned8'][gen_data['problem_id'].isin(sampled_gen_data.problem_id)]

In [7]:
sampled_hum_data['gpt_resample_code_cleand3'] = hum_data['gpt_resample_code_cleaned3'][hum_data['problem_id'].isin(sampled_hum_data.problem_id)]

sampled_hum_data['gpt_resample_code_cleand4'] = hum_data['gpt_resample_code_cleaned4'][hum_data['problem_id'].isin(sampled_hum_data.problem_id)]

sampled_hum_data['gpt_resample_code_cleand5'] = hum_data['gpt_resample_code_cleaned5'][hum_data['problem_id'].isin(sampled_hum_data.problem_id)]

sampled_hum_data['gpt_resample_code_cleand6'] = hum_data['gpt_resample_code_cleaned6'][hum_data['problem_id'].isin(sampled_hum_data.problem_id)]

sampled_hum_data['gpt_resample_code_cleand7'] = hum_data['gpt_resample_code_cleaned7'][hum_data['problem_id'].isin(sampled_hum_data.problem_id)]

sampled_hum_data['gpt_resample_code_cleand8'] = hum_data['gpt_resample_code_cleaned8'][hum_data['problem_id'].isin(sampled_hum_data.problem_id)]

KeyError: 'gpt_resample_code_cleand5'

In [42]:
sampled_hum_data.to_pickle("dataset/apps_hum_test100.pkl")
sampled_gen_data.to_pickle("dataset/apps_gen_test100.pkl")